In [1]:
# Import libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt 
#import hvplot.xarray
import warnings

# these libraries help reading cloud data
import fsspec 
import s3fs
import requests

warnings.simplefilter("ignore")  # filter some warning messages
xr.set_options(display_style="html",keep_attrs=True)  

In [3]:
from earthdata import Auth 
auth = Auth().login()

Enter your Earthdata Login username:  cgentemann
Enter your Earthdata password:  ··········


You're now authenticated with NASA Earthdata Login


In [ ]:
def begin_s3_direct_access():
    url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})

In [4]:
start_year = 2002
end_year = 2002
print(start_year, end_year)

2002 2002


In [19]:
lyr,idyjl=2003,1
date = (np.datetime64(str(lyr)+'-01-01')+np.timedelta64(idyjl-1,'D')).astype(object)
def make_full_path(date):
    dstr = str(date.year).zfill(2)+str(date.month).zfill(2)+str(date.day).zfill(2)
    #https://mur-sst.s3.us-west-2.amazonaws.com/
    #c_data <- s3read_using(FUN = nc_open, object = "20040111090000-JPL-L4_GHRSST-SSTfnd-MUR25-GLOB-v02.0-fv04.2.nc", bucket="podaac-ops-cumulus-protected/MUR25-JPL-L4-GLOB-v04.2" )
    #return f"https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20030102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
    return f"https://podaac-ops-cumulus-public.us-west-2.amazonaws.com/MUR-JPL-L4-GLOB-v4.1/{dstr}090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
    #return f"https://podaac-ops-cumulus-public.us-west-2.aws./MUR-JPL-L4-GLOB-v4.1/{dstr}090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"
ss = make_full_path(date)
ss

'https://podaac-ops-cumulus-public.us-west-2.amazonaws.com/MUR-JPL-L4-GLOB-v4.1/20030101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

In [21]:
#fs = fsspec.filesystem('s3', anon=False)  
#infile = fs.open(ss)
ds = xr.open_dataset(ss)
ds

OSError: [Errno -68] NetCDF: I/O failure: b'https://podaac-ops-cumulus-public.us-west-2.amazonaws.com/MUR-JPL-L4-GLOB-v4.1/20030101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

In [ ]:
def make_full_path(variable, time):
    return f"http://data-provider.org/data/{variable}/{variable}_{time:02d}.txt"

# check that it works
make_full_path("humidity", 3)